# Questão 1) Coleta de dados

Instalação das bibliotecas usadas

In [ ]:
!pip install pandas
!pip install requests
!pip install pyperclip


## Baixa dados sobre Covid da web

O código utiliza a biblioteca ```requests```, mas a primeira célula já executa o comando ```!pip install requests```.

In [10]:
import requests
import os

def download_file(id, destination):
    URL = "https://drive.google.com/uc?id=1oiBDNZD65P3OCmOc1xrOlhL391MDcm-o&export=download"

    session = requests.Session()
    response = session.get(URL, params={'id': id})
    token = get_confirm_token(response)
    if(token):
        params = {'id': id, 'confirm': token}
        response = session.get(URL, params=params)
    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

# https://stackoverflow.com/questions/30582162/creating-a-missing-directory-file-structure-python
def createAndOpen(filename, mode):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    return open(filename, mode)

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with createAndOpen(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

# Endereço do arquivo CSV
endereco_arquivo_dados = os.path.join(os.getcwd(), 'DW-trabalho-workspace', 'DW_trabalho_final_grupo3', 'data', 'dados_covid.csv')

if __name__ == "__main__":
    file_id = '1oiBDNZD65P3OCmOc1xrOlhL391MDcm-o'
    destination = endereco_arquivo_dados
    #destination = 'dados_covid.csv'
    download_file(file_id, destination)

# Questão 3) Cria banco de dados

In [ ]:
import sqlite3
from sqlite3 import Error
import os

#https://www.sqlitetutorial.net/sqlite-python/


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


def getPath():
    return os.path.join(os.getcwd(), 'dw_prova_grupo.db')

if __name__ == '__main__':
    print(getPath())
    conn = create_connection(getPath())

In [ ]:

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


def getPath():
    return os.path.join(os.getcwd(), 'dw_prova_grupo.db')

if __name__ == '__main__':
    print(getPath())
    conn = create_connection(getPath())

## Cria tabela estrela

In [13]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

In [14]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [15]:
def main():
    database = getPath()

    #cria tabela fato
    sql_create_vacinacao_covid_brasil = """ CREATE TABLE IF NOT EXISTS vacinacao_covid_brasil (
                                                fk_paciente_id text,
                                                fk_vacina_codigo integer,
                                                fk_estabelecimento_id integer,
                                                fk_tempo_id integer,
                                                FOREIGN KEY (fk_paciente_id) REFERENCES paciente(paciente_id),
                                                FOREIGN KEY (fk_vacina_codigo) REFERENCES vacina(vacina_codigo),
                                                FOREIGN KEY (fk_estabelecimento_id) REFERENCES estabelecimento(estabelecimento_id),
                                                FOREIGN KEY (fk_tempo_id) REFERENCES tempo(tempo_id),
                                                sistema_origem text,
                                                id_sistema_origem integer
                                            );"""

    #cria tabelas dimensao
    sql_create_tempo_table = """ CREATE TABLE IF NOT EXISTS tempo (
                                    tempo_id integer PRIMARY KEY AUTOINCREMENT,
                                    vacina_dataaplicacao text,
                                    paciente_datanascimento text,
                                    data_importacao_rnds text
                                );"""

    sql_create_paciente_table = """ CREATE TABLE IF NOT EXISTS paciente (
                                        paciente_id text PRIMARY KEY,
                                        document_id text,
                                        paciente_idade integer,
                                        paciente_enumsexobiologico char,
                                        paciente_racacor_valor text,
                                        paciente_racacor_codigo integer,
                                        paciente_endereco_coibgemunicipio integer,
                                        paciente_endereco_copais integer,
                                        paciente_endereco_nmmunicipio text,
                                        paciente_endereco_nmpais text,
                                        paciente_endereco_uf text,
                                        paciente_endereco_cep text,
                                        paciente_nacionalidade_enumnacionalidade char
                                    );"""

    sql_create_vacina_table = """ CREATE TABLE IF NOT EXISTS vacina (
                                    vacina_codigo integer PRIMARY KEY AUTOINCREMENT,
                                    vacina_nome text,
                                    vacina_numdose integer,
                                    vacina_descricao_dose text,
                                    vacina_fabricante_referencia text,
                                    vacina_fabricante_nome text,
                                    vacina_lote text,
                                    vacina_categoria_nome text,
                                    vacina_categoria_codigo integer,
                                    vacina_grupoatendimento_nome text,
                                    vacina_grupoatendimento_codigo integer
                                );"""

    sql_create_estabelecimento_table = """ CREATE TABLE IF NOT EXISTS estabelecimento (
                                            estabelecimento_id integer PRIMARY KEY AUTOINCREMENT,
                                            estabelecimento_municipio_nome text,
                                            estabelecimento_municipio_codigo integer,
                                            estabelecimento_valor integer,
                                            estabelecimento_nofantasia text,
                                            estabelecimento_razaosocial text,
                                            estabelecimento_uf text
                                        );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create historico table
        create_table(conn, sql_create_vacinacao_covid_brasil)
        create_table(conn, sql_create_paciente_table)
        create_table(conn, sql_create_vacina_table)
        create_table(conn, sql_create_estabelecimento_table)
        create_table(conn, sql_create_tempo_table)
    else:
        print("Error! cannot create the database connection.")

    #conn.commit()
    #conn.close()

In [16]:
if __name__ == '__main__':
    main()

near "sistema_origem": syntax error


# Questão 4) Carga de dados

## Insere dados na tabela

O código utiliza a biblioteca ```pandas```, mas a primeira célula já executa o comando ```!pip install pandas```.

In [17]:
import pandas as pd

#function that insert csv data into database
def insert_data(df, table_name):
    conn = create_connection(getPath())
    df.to_sql(table_name, conn, if_exists='append', index=False)

# Colunas com os dados da tabela paciente
paciente_atributos = ['document_id','paciente_idade','paciente_enumsexobiologico','paciente_racacor_valor','paciente_racacor_codigo','paciente_endereco_coibgemunicipio','paciente_endereco_copais','paciente_endereco_nmmunicipio','paciente_endereco_nmpais','paciente_endereco_uf','paciente_endereco_cep','paciente_nacionalidade_enumnacionalidade']

# Colunas com os dados da tabela vacina
vacina_atributos = ['vacina_nome','vacina_numdose','vacina_descricao_dose','vacina_fabricante_referencia','vacina_fabricante_nome','vacina_lote','vacina_categoria_nome','vacina_categoria_codigo','vacina_grupoatendimento_nome','vacina_grupoatendimento_codigo'] 

# Colunas com os dados da tabela estabelecimento
estabelecimento_atributos = ['estabelecimento_municipio_nome', 'estabelecimento_municipio_codigo', 'estabelecimento_valor', 'estabelecimento_razaosocial', 'estabelecimento_uf']

# Colunas com os dados da tabela tempo
tempo_atributos = ['vacina_dataaplicacao', 'paciente_datanascimento', 'data_importacao_rnds']

# Colunas com os dados da tabela fato
tabela_fato_atributos = ['sistema_origem','id_sistema_origem']

# Le os dados do csv e insere no banco de dados
# Insere os dados na tabela paciente
dados_paciente = pd.read_csv(endereco_arquivo_dados, usecols = paciente_atributos, sep = ';')
insert_data(dados_paciente, 'paciente')

# Insere os dados na tabela vacina
dados_vacina = pd.read_csv(endereco_arquivo_dados, usecols = vacina_atributos, sep = ';')
insert_data(dados_vacina, 'vacina')

# Insere os dados na tabela estabelecimento
dados_estabelecimento = pd.read_csv(endereco_arquivo_dados, usecols = estabelecimento_atributos, sep = ';')
insert_data(dados_estabelecimento, 'estabelecimento')

# Insere os dados na tabela tempo
dados_tempo = pd.read_csv(endereco_arquivo_dados, usecols = tempo_atributos, sep=';')
insert_data(dados_tempo, 'tempo')

# Insere os dados na tabela fato
dados_fato = pd.read_csv(endereco_arquivo_dados, usecols = tabela_fato_atributos, sep = ';')
insert_data(dados_fato, 'vacinacao_covid_brasil')

# Questão 5) Análise de dados

## IMPORTANTE: Assistir o seguinde vídeo: https://www.youtube.com/watch?v=wjvb_x1I4rI

Para executar o workflow KNIME de maneira correta, deve-se assistir vídeo acima para entender a função das 2 células abaixo

Os códigos abaixo utilizam a biblioteca ```pyperclip```, mas a primeira célula já executa o comando ```!pip install pyperclip```.

O método pyperclip.copy() serve para copiar uma string para área de transferência do computador
A célula abaixo pega o endereço que será usado para o Workspace do KNIME

In [18]:
import pyperclip
import os

def getWorkflowPath():
    knWorkspacePath = os.path.join(os.getcwd(), 'DW-trabalho-workspace')
    print(knWorkspacePath)
    return knWorkspacePath

knwfPath = getWorkflowPath()
pyperclip.copy(knwfPath)

e:\UFRJ\Eletivas\Data Warehouse no Suporte a Tomada de Decisão\Exercícios em grupo\dw_trabalho_final_grupo\DW-trabalho-workspace


A célula abaixo pega o endereço que será usado para um dos passos no KNIME
O endereço será usado na exportação do Workflow do KNIME, que é essencial para que o KNIME consiga ler da pasta correta

In [19]:
import pyperclip
import os

def getWorkflowPath():
    knwfPath = os.path.join(os.getcwd(), 'DW-trabalho-workspace', 'DW_trabalho_final_grupo3.knwf')
    print(knwfPath)
    return knwfPath

knwfPath = getWorkflowPath()
pyperclip.copy(knwfPath)

e:\UFRJ\Eletivas\Data Warehouse no Suporte a Tomada de Decisão\Exercícios em grupo\dw_trabalho_final_grupo\DW-trabalho-workspace\DW_trabalho_final_grupo3.knwf
